In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

Using TensorFlow backend.


In [2]:
np.random.seed(7)

In [3]:
# number of features used in the regression
window_size = 10
#


# lstm_num_timesteps
lstm_num_timesteps = 10
# whether to use LSTM or MLP
use_LSTM = True
# lstm_num_features
lstm_num_features = 1
# stateful?
lstm_stateful = False
# use two lstm layers?
lstm_stack_layers = False


batch_size = 1
num_epochs = 2
num_neurons = 4

# scale the dataset to values between scale_min and scale_max
scale = True
scale_min = -1
scale_max = 1

In [4]:
# load the dataset
#df_train = pd.read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
df_train = pd.read_csv('108_7_3.train.csv', usecols=[1], engine='python', skipfooter=3)
ts_train = df_train.values.astype('float64')
df_test = pd.read_csv('108_7_3.test.csv', usecols=[1], engine='python', skipfooter=3)
ts_test = df_test.values.astype('float64')
ts_all = np.append(ts_train, ts_test).reshape(-1,1)
len_overall = len(ts_all)

In [5]:
#for testing
#ts_train = np.arange(1,101, dtype='float64').reshape(-1,1)  
#ts_test = np.arange(101,121, dtype='float64').reshape(-1,1)
#window_size = 5
#ts_all = np.append(ts_train, ts_test).reshape(-1,1)
#len_overall = len(ts_all)

In [6]:
len_overall

983

In [7]:
ts_all.dtype

dtype('float64')

In [8]:
ts_train.shape, ts_test.shape

((896, 1), (87, 1))

In [9]:
ts_train[:10]

array([[ 4164.],
       [ 3540.],
       [ 3560.],
       [ 3094.],
       [ 3658.],
       [ 3049.],
       [ 3526.],
       [ 3290.],
       [ 4447.],
       [ 2305.]])

In [10]:
ts_test[:10]

array([[ 1131.],
       [  816.],
       [ 1084.],
       [ 1283.],
       [ 1363.],
       [ 1060.],
       [ 1272.],
       [  636.],
       [ 1095.],
       [ 1284.]])

In [11]:
if scale:
    scaler = MinMaxScaler(feature_range=(scale_min, scale_max))
    ts_train = scaler.fit_transform(ts_train)
    ts_test = scaler.transform(ts_test)

In [12]:
scaler.data_min_, scaler.data_max_

(array([ 322.]), array([ 4749.]))

In [13]:
ts_train[:10]

array([[ 0.73571267],
       [ 0.45380619],
       [ 0.46284165],
       [ 0.25231534],
       [ 0.50711543],
       [ 0.23198554],
       [ 0.44748136],
       [ 0.34086289],
       [ 0.86356449],
       [-0.10413372]])

In [14]:
ts_test[:10]

array([[-0.63451547],
       [-0.77682403],
       [-0.65574881],
       [-0.56584595],
       [-0.52970409],
       [-0.66659137],
       [-0.57081545],
       [-0.85814321],
       [-0.65077931],
       [-0.56539417]])

In [15]:
# split into train and test sets
#train_size = int(len(ts) * 0.67)
#test_size = len(ts) - train_size
#ts_train, ts_test = ts[0:train_size,:], ts[train_size:len(ts),:]
#print(len(ts_train), len(ts_test))


In [16]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        a = dataset[i:(i + window_size), 0]
        dataX.append(a)
        dataY.append(dataset[i + window_size, 0])
    return np.array(dataX), np.array(dataY)

In [17]:
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df = df.drop(0)
	return df

In [18]:
X_train, y_train = create_dataset(ts_train, window_size)
X_test, y_test = create_dataset(ts_test, window_size)
# the train and test matrices end up shorter than the respective timeseries by window_size + 1!
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((886, 10), (77, 10), (886,), (77,))

In [19]:
X_train[:5,:]

array([[ 0.73571267,  0.45380619,  0.46284165,  0.25231534,  0.50711543,
         0.23198554,  0.44748136,  0.34086289,  0.86356449, -0.10413372],
       [ 0.45380619,  0.46284165,  0.25231534,  0.50711543,  0.23198554,
         0.44748136,  0.34086289,  0.86356449, -0.10413372,  0.05489044],
       [ 0.46284165,  0.25231534,  0.50711543,  0.23198554,  0.44748136,
         0.34086289,  0.86356449, -0.10413372,  0.05489044,  0.11226564],
       [ 0.25231534,  0.50711543,  0.23198554,  0.44748136,  0.34086289,
         0.86356449, -0.10413372,  0.05489044,  0.11226564,  0.24056923],
       [ 0.50711543,  0.23198554,  0.44748136,  0.34086289,  0.86356449,
        -0.10413372,  0.05489044,  0.11226564,  0.24056923,  0.6778857 ]])

In [20]:
y_train[:5]

array([ 0.05489044,  0.11226564,  0.24056923,  0.6778857 ,  0.17957985])

In [21]:
X_test[:5,:]

array([[-0.63451547, -0.77682403, -0.65574881, -0.56584595, -0.52970409,
        -0.66659137, -0.57081545, -0.85814321, -0.65077931, -0.56539417],
       [-0.77682403, -0.65574881, -0.56584595, -0.52970409, -0.66659137,
        -0.57081545, -0.85814321, -0.65077931, -0.56539417, -0.62909419],
       [-0.65574881, -0.56584595, -0.52970409, -0.66659137, -0.57081545,
        -0.85814321, -0.65077931, -0.56539417, -0.62909419, -0.68601762],
       [-0.56584595, -0.52970409, -0.66659137, -0.57081545, -0.85814321,
        -0.65077931, -0.56539417, -0.62909419, -0.68601762, -0.55590693],
       [-0.52970409, -0.66659137, -0.57081545, -0.85814321, -0.65077931,
        -0.56539417, -0.62909419, -0.68601762, -0.55590693, -0.73480913]])

In [22]:
y_test[:5]

array([-0.62909419, -0.68601762, -0.55590693, -0.73480913, -0.84730066])

In [23]:
if use_LSTM:
    # reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
    X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))

In [25]:
model = Sequential()

if use_LSTM:
    
    # the last state for each sample at index i in a batch will be used as initial state
    # for the sample of index i in the following batch
    if lstm_stateful:
        #
        #
        if lstm_stack_layers:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True,
                       return_sequences = True))
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True))
        # 
        else:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True))
          
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        
        train_rmse, test_rmse = list(), list()
        for i in range(num_epochs):
            # shuffle must be False!
            model.fit(X, y, epochs=1, batch_size = batch_size, verbose = 0, shuffle = False)
            model.reset_states()
            # evaluate model on train data
            #raw_train = raw[-(len(train)+len(test)+1):-len(test)]
            #train_rmse.append(evaluate(model, raw_train, train, scaler, 0, batch_size))
            #model.reset_states()
            # evaluate model on test data
            #raw_test = raw[-(len(test)+1):]
            #test_rmse.append(evaluate(model, raw_test, test, scaler, 0, batch_size))
            #model.reset_states()
        history = DataFrame()
        history['train'], history['test'] = train_rmse, test_rmse
        
        print(history)
        
    # stateful == False    
    else:
        
        
        if lstm_stack_layers:
            # input_dim: dimensionality of the input (alternatively, input_shape)
            # required when using this layer as the first layer in a model
            model.add(LSTM(num_neurons, input_dim = lstm_num_features, return_sequences = True)
            model.add(LSTM(num_neurons))
        # 
        else:
            model.add(LSTM(num_neurons, input_dim = lstm_num_features, return_sequences = True)
        
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)
        
   

# feedforward
else:
    
    model.add(Dense(num_neurons, input_dim = window_size, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)


Epoch 1/2
886/886 [==============================] - 11s - loss: 0.0396    
Epoch 2/2
886/886 [==============================] - 11s - loss: 0.0334    


In [26]:
model.evaluate(X_test, y_test, batch_size = X_test.shape[0])

77/77 [==============================] - 0s


0.018286425620317459

In [ ]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [ ]:
y_train[:10]

In [ ]:
pred_train[:10,0]

In [ ]:
y_test[:10]

In [ ]:
pred_test[:10,0]

In [ ]:
if scale:
    pred_train = scaler.inverse_transform(pred_train)
    y_train = scaler.inverse_transform(y_train.reshape(-1,1))
    pred_test = scaler.inverse_transform(pred_test)
    y_test = scaler.inverse_transform(y_test.reshape(-1,1))


In [ ]:
y_train[:10],pred_train[:10,0]

In [ ]:
y_test[:10],pred_test[:10,0]

In [ ]:
# calculate root mean squared error
rsme_train = math.sqrt(mean_squared_error(y_train, pred_train[:,0]))
print('Train Score: %.2f RMSE' % (rsme_train))
rsme_test = math.sqrt(mean_squared_error(y_test, pred_test[:,0]))
print('Test Score: %.2f RMSE' % (rsme_test))

In [ ]:
print(len(ts_train), len(pred_train), len(y_train))
len(ts_test), len(pred_test), len(y_test) 

In [ ]:
# shift train predictions for plotting
pred_train_shifted = np.empty_like(ts_all)
print(pred_train_shifted.size)
pred_train_shifted[:, :] = np.nan
# train predictions start at position window_size + 1 (or window_size, if counting from 0)
pred_train_shifted[window_size : len(pred_train) + window_size, :] = pred_train
pred_train_shifted[:13]

In [ ]:
# shift test predictions for plotting
pred_test_shifted = np.empty_like(ts_all)
pred_test_shifted[:, :] = np.nan
pred_test_shifted[len(pred_train) + (window_size * 2) : len_overall + 1, :] = pred_test
pred_test_shifted[-13:]

In [ ]:
plt.plot(ts_all)
plt.plot(pred_train_shifted)
plt.plot(pred_test_shifted)
plt.show()

In [ ]:
plot_start = -100
plot_end = -1
plt.plot(ts_all[plot_start:plot_end])
plt.plot(pred_train_shifted[plot_start:plot_end])
plt.plot(pred_test_shifted[plot_start:plot_end])
plt.show()